In [ ]:
import numpy as np
import pandas as pd
import requests
from datetime import date
import time
# 1. Defino el bounding box y el mallado a 0.1°
def obtener_clima(lugar,inicio):
    if lugar=="Villa María":
        min_lat, max_lat = -32.5, -32.3
        min_lon, max_lon = -63.4, -63.1
    elif lugar=="Argentina":
        min_lat, max_lat = -55.0, -21.7
        min_lon, max_lon = -73.5, -53.6

    lats = np.arange(min_lat, max_lat + 1e-6, 0.5)
    lons = np.arange(min_lon, max_lon + 1e-6, 0.5)

    # 2. Fechas
    start_date = inicio
    end_date   = date.today().isoformat()  # e.g. "2025-07-07"

    # 3. Variables diarias correctas
    daily_vars = [
        "temperature_2m_mean",
        "temperature_2m_min",
        "temperature_2m_max",
        "precipitation_sum",
        "relative_humidity_2m_mean",
        "wind_speed_10m_mean"
    ]
    daily = ",".join(daily_vars)

    # 4. Función que pide un punto y devuelve un DataFrame
    def fetch_point(lat, lon):
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude":      lat,
            "longitude":     lon,
            "start_date":    start_date,
            "end_date":      end_date,
            "daily":         daily,
            "timezone":      "auto",
            "model":         "era5-land",
            "cell_selection":"nearest"
        }
        resp = requests.get(url, params=params)

        
        resp.raise_for_status()
        js = resp.json()
        # la API devuelve un dict con 'daily': {... arrays ...}
        df = pd.DataFrame(js["daily"])
        df["lat"] = lat
        df["lon"] = lon
        return df

    # 5. Itero sobre la grilla y concateno
    all_dfs = []
    total = len(lats) * len(lons)
    i = 0
    for lat in lats:
        for lon in lons:
            i += 1
            print(f"[{i}/{total}] solicitando {lat:.2f}, {lon:.2f} ...", end=" ")
            try:
                df_pt = fetch_point(lat, lon)
                all_dfs.append(df_pt)
                print("✔")
            except requests.HTTPError as e:
                print(f"[ERROR] {e.response.status_code}")
            time.sleep(2)
    # 6. Uno todo en un único DataFrame
    if all_dfs:
        df_all = pd.concat(all_dfs, ignore_index=True)
        print("Registros totales:", len(df_all))
    else:
        df_all = pd.DataFrame()
        print("No se obtuvieron datos.")

    # 7. Ejemplo de pivot para dejar un DataFrame ancho:
    #    lat, lon, date, temp_mean, temp_min, temp_max, precip, rh_mean, wind_mean
    df_all = df_all.rename(columns={
        "temperature_2m_mean":     "t_mean",
        "temperature_2m_min":      "t_min",
        "temperature_2m_max":      "t_max",
        "precipitation_sum":       "precip_sum",
        "relative_humidity_2m_mean":"rh_mean",
        "wind_speed_10m_mean":     "wind_mean"
    })
    return df_all


# df_villa=obtener_clima("Villa María","2023-08-01")
time.sleep(30*60)
df_argentina=obtener_clima("Argentina","2023-08-01")



[1/2680] solicitando -55.00, -73.50 ... ✔
[2/2680] solicitando -55.00, -73.00 ... ✔
[3/2680] solicitando -55.00, -72.50 ... ✔
[4/2680] solicitando -55.00, -72.00 ... ✔
[5/2680] solicitando -55.00, -71.50 ... ✔
[6/2680] solicitando -55.00, -71.00 ... ✔
[7/2680] solicitando -55.00, -70.50 ... ✔
[8/2680] solicitando -55.00, -70.00 ... ✔
[9/2680] solicitando -55.00, -69.50 ... ✔
[10/2680] solicitando -55.00, -69.00 ... ✔
[11/2680] solicitando -55.00, -68.50 ... ✔
[12/2680] solicitando -55.00, -68.00 ... ✔
[13/2680] solicitando -55.00, -67.50 ... ✔
[14/2680] solicitando -55.00, -67.00 ... ✔
[15/2680] solicitando -55.00, -66.50 ... ✔
[16/2680] solicitando -55.00, -66.00 ... ✔
[17/2680] solicitando -55.00, -65.50 ... ✔
[18/2680] solicitando -55.00, -65.00 ... ✔
[19/2680] solicitando -55.00, -64.50 ... ✔
[20/2680] solicitando -55.00, -64.00 ... ✔
[21/2680] solicitando -55.00, -63.50 ... ✔
[22/2680] solicitando -55.00, -63.00 ... ✔
[23/2680] solicitando -55.00, -62.50 ... ✔
[24/2680] solicitand

In [ ]:
import numpy as np
import xarray as xr


def interpolar(df_all):
    # 1. Partimos del DataFrame `df_all` con columnas:
    #    ['time', 'lat', 'lon', 't_mean', 't_min', 't_max', 'precip_sum', 'rh_mean', 'wind_mean']

    # 2. Convertimos a xarray Dataset
    ds = (
        df_all
        .set_index(['time', 'lat', 'lon'])
        .to_xarray()
    )

    # 3. Definimos la nueva grilla a 0.01°
    min_lat, max_lat = float(df_all.lat.min()), float(df_all.lat.max())
    min_lon, max_lon = float(df_all.lon.min()), float(df_all.lon.max())
    new_lats = np.arange(min_lat, max_lat + 1e-8, 0.001)
    new_lons = np.arange(min_lon, max_lon + 1e-8, 0.001)

    # 4. Interpolamos espacialmente (método lineal)
    ds_fine = ds.interp(
        lat=new_lats,
        lon=new_lons,
        method="linear"
    )

    # 5. (Opcional) Convertir de vuelta a DataFrame
    df_fine = (
        ds_fine
        .to_dataframe()
        .reset_index()
    )

    # Ahora `df_fine` contiene tus variables diarias interpoladas en la grilla 0.01° × 0.01°
    print(df_fine.head())
    return(df_fine)

         time   lat     lon  t_mean   t_min   t_max  precip_sum  rh_mean  \
0  2023-08-01 -32.5 -63.400  21.300  16.900  29.000         0.0    74.00   
1  2023-08-01 -32.5 -63.399  21.302  16.899  29.001         0.0    73.99   
2  2023-08-01 -32.5 -63.398  21.304  16.898  29.002         0.0    73.98   
3  2023-08-01 -32.5 -63.397  21.306  16.897  29.003         0.0    73.97   
4  2023-08-01 -32.5 -63.396  21.308  16.896  29.004         0.0    73.96   

   wind_mean  
0     10.500  
1     10.502  
2     10.504  
3     10.506  
4     10.508  


In [ ]:
len(df_fine["precip_sum"].unique())

14863139